<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Algoritmo WalkSAT</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivo <a class="anchor" id="inicio"></a>

En clase hemos visto el algoritmo walkSAT para encontrar un modelo $I$ para una fórmula $S$ en forma clausal. En este notebook implementaremos este algoritmo en Python.

Desarrollaremos la explicación mediante las siguientes secciones.

# Secciones

1. [Funciones auxiliares](#aux)
2. [Walksat](#walk)
3. [Comparación de tiempos](#comp)


# Funciones auxiliares <a class="anchor" id="aux"></a>

([Volver al inicio](#inicio))

Para poder implementar el algoritmo walkSAT requerimos algunas funciones auxiliares:

* `interpretacion_aleatoria(letrasp)`: Devuelve una interpretación seleccionando aleatoriamente el valor True o False para cada letra proposicional en `letrasp`. 

`flip_literal`$(I, \ell)$: Modifica el valor de verdad asignado por la interpretación $I$ al literal $\ell$.

In [14]:
from random import randint
from copy import deepcopy

def interpretacion_aleatoria(letrasp):
    I = {p:randint(0,1)==1 for p in letrasp}
    return I

def flip_literal(I, l):
    p = l[-1]
    valor = False if I[p] else True
    Ip = deepcopy(I)
    Ip[p] = valor
    return Ip

In [15]:
letrasp = ['p', 'q', 'r']
I = interpretacion_aleatoria(letrasp)
I

{'p': False, 'q': False, 'r': False}

In [16]:
I = flip_literal(I, 'p')
I

{'p': True, 'q': False, 'r': False}

Para manejar la información sobre las letras proposicionales y las cláusulas de la fórmula que vayamos a trabajar, hemos definido una clase `WalkSatEstado` de la siguiente manera:

In [17]:
class WalkSatEstado():
    
    def __init__(self, S):
        self.S = S
        self.letrasp = list(set([l[-1] for C in self.S for l in C]))
        self.I = interpretacion_aleatoria(self.letrasp)
        self.I_lits = set([p for p in self.letrasp if self.I[p]] + ['-'+p for p in self.letrasp if not self.I[p]])
        self.clausulas_sat = [C for C in self.S if any((True for x in self.I_lits if x in C))]
        self.clausulas_unsat = [C for C in self.S if C not in self.clausulas_sat]

    def actualizar(self, I):
        self.I = I
        self.I_lits = set([p for p in self.letrasp if self.I[p]] + ['-'+p for p in self.letrasp if not self.I[p]])
        self.clausulas_sat = [C for C in self.S if any((True for x in self.I_lits if x in C))]
        self.clausulas_unsat = [C for C in self.S if C not in self.clausulas_sat]
       
    def SAT(self):
        return len(self.clausulas_unsat) == 0

    def break_count(self, l):
        clausulas_break_count = [C for C in self.clausulas_sat if set(C).intersection(self.I_lits)==set([l])]
        return len(clausulas_break_count)

Consideremos una fórmula en forma clausal y veamos el correspondiente objeto `WalkSatEstado`:

In [18]:
S = [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
w = WalkSatEstado(S)
print("Fórmula:", S)
print("Letras proposicionales:", w.letrasp)
print("Interpretación:", w.I)
print("Literales determinados por la interpretación:", w.I_lits)
print("Cláusulas sat:", w.clausulas_sat)
print("Cláusulas unsat", w.clausulas_unsat)
print("I hace verdadera la fórmula?:", w.SAT())

Fórmula: [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
Letras proposicionales: ['p', 'r', 'q']
Interpretación: {'p': False, 'r': True, 'q': True}
Literales determinados por la interpretación: {'-p', 'r', 'q'}
Cláusulas sat: [['r'], ['q', '-r'], ['-p', '-q', 'r']]
Cláusulas unsat [['p', '-r']]
I hace verdadera la fórmula?: False


Observe que esta clase tiene definido el método `break_count()` para determinar cuántas cláusulas sat se vuelven unsat si se voltea el valor de un literal:

In [19]:
w.break_count('-r')

0

---

# WalkSAT <a class="anchor" id="walk"></a>

([Volver al inicio](#inicio))

Presentamos ahora el algoritmo walkSAT.

<img src="./img/walksat.png" width="auto"/>

**Ejercicio 1:**

Implemente la función `walkSAT`.

Pruebe su respuesta con los siguientes ejemplos:

1. `S = [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]` 

Solución:  $I\,{=}\,\{r: True, p: True, q: True\}$.

2. `S=[['s'], ['-r', '-q'], ['r', 'q'], ['p', '-s'], ['r', '-s'], ['-p', '-r', 's']]` 

Solución: $I\,{=}\,\{s: True, p: True, r: True, q: False\}$.

3. `S = [['s'], ['p','-t'], ['q','-t'], ['-p','-q','t'], ['t','-s'], ['r','-s'], ['-t','-r','s']]` 

Solución: $I\,{=}\,\{s: True, t: True, p: True, q: True, r: True\}$.

4. `S = [['p','-q'],['-p','-q'],['q','r'],['-q''-r'],['-p','-r'],['p','-r']]` 

Solución: Intento fallido.

In [143]:
from random import choice, uniform

def walkSAT(S):
    w =  WalkSatEstado(S)
    max_flips = 100
    max_tries = 100
    p = uniform(0, 1)
    for i in range (max_tries):
        for j in range(max_flips):
            w.actualizar(w.I)
            if(len(w.clausulas_unsat) == 0):
                return "Satisfacible", w.I
            C = choice(w.clausulas_unsat)
            bk = {}
            flag = 0
            for x in C:
                break_count = w.break_count(x)
                if break_count == 0:
                    v = x
                    flag = 1
                    break
                else:
                    bk[x] = break_count
                    
            if flag != 1:
                if uniform(0, 1) <= p:
                    v = choice(C)
                else:
                    MAX = bk[C[0]]
                    v = C[0]
                    for x in C:
                        if(bk[x]<MAX):
                            MAX = bk[x]
                            v = x
            w.I = flip_literal(w.I, v)
    return "intento fallido"

In [144]:
S = [['r'], ['p', '-r'], ['q', '-r'], ['-p', '-q', 'r']]
walkSAT(S)

('Satisfacible', {'p': True, 'r': True, 'q': True})

In [146]:
S = [['s'], ['-r', '-q'], ['r', 'q'], ['p', '-s'], ['r', '-s'], ['-p', '-r', 's']]
walkSAT(S)

('Satisfacible', {'p': True, 's': True, 'r': True, 'q': False})

In [147]:
S = [['s'], ['p','-t'], ['q','-t'], ['-p','-q','t'], ['t','-s'], ['r','-s'], ['-t','-r','s']]
walkSAT(S)

('Satisfacible', {'s': True, 'r': True, 'q': True, 'p': True, 't': True})

In [153]:
S = [['p','-q'],['-p','-q'],['q','r'],['-q','-r'],['-p','-r'],['p','-r']]
walkSAT(S)

'intento fallido'

---

# Comparación de tiempos <a class="anchor" id="comp"></a>

([Volver al inicio](#inicio))

Comparamos los tiempos de búsqueda de modelo para fórmulas aleatorias cada vez más grandes, entre los algoritmos DPLL y walkSAT.

**Ejercicio 2:**

Incluya aquí su implementación del DPLL.

In [164]:
from Logica import *

def inorder_to_tree(cadena:str):
    conectivos = ['Y', 'O', '>', '=']
    if len(cadena) == 1:
        return Letra(cadena)
    elif cadena[0] == '-':
        return Negacion(inorder_to_tree(cadena[1:]))
    elif cadena[0] == "(":
        counter = 0 #Contador de parentesis
        for i in range(1, len(cadena)):
            if cadena[i] == "(":
                counter += 1
            elif cadena[i] == ")":
                counter -=1
            elif cadena[i] in conectivos and counter == 0:
                return Binario(cadena[i], inorder_to_tree(cadena[1:i]),inorder_to_tree(cadena[i + 1:-1]))
    else:
        raise Exception('¡Cadena inválida!')

def tseitin(A):
    '''
    Algoritmo de transformacion de Tseitin
    Input: A (cadena) en notacion inorder
    Output: B (cadena), Tseitin
    '''
    # Creamos letras proposicionales nuevas
    f = inorder_to_tree(A)
    letrasp = f.letras()
    cods_letras = [ord(x) for x in letrasp]
    m = max(cods_letras) + 256
    letrasp_tseitin = [chr(x) for x in range(m, m + f.num_conec())]
    letrasp = list(letrasp) + letrasp_tseitin
    L = [] # Inicializamos lista de conjunciones
    Pila = [] # Inicializamos pila
    i = -1 # Inicializamos contador de variables nuevas
    s = A[0] # Inicializamos símbolo de trabajo
    while len(A) > 0: # Recorremos la cadena
        # print("Pila:", Pila, " L:", L, " s:", s)
        if (s in letrasp) and (len(Pila) > 0) and (Pila[-1]=='-'):
            i += 1
            atomo = letrasp_tseitin[i]
            Pila = Pila[:-1]
            Pila.append(atomo)
            L.append(atomo + "=-" + s)
            A = A[1:]
            if len(A) > 0:
                s = A[0]
        elif s == ')':
            w = Pila[-1]
            O = Pila[-2]
            v = Pila[-3]
            Pila = Pila[:len(Pila)-4]
            i += 1
            atomo = letrasp_tseitin[i]
            L.append(atomo + "=(" + v + O + w + ")")
            s = atomo
        else:
            Pila.append(s)
            A = A[1:]
            if len(A) > 0:
                s = A[0]
    if i < 0:
        atomo = Pila[-1]
    else:
        atomo = letrasp_tseitin[i]
    B = [[[atomo]]] + [a_clausal(x) for x in L]
    B = [val for sublist in B for val in sublist]
    return B

def dpll(S, I):
    S, I = unit_propagate(S, I)
    if [] in S:
        return "Insatisfacible", {}
    if len(S) == 0:
        return "Satisfacible", I    
    l = choice(choice(S))
    lc = complemento(l)
    newS = eliminar_literal(S, l)
    newI = extender_I(I, l)
    sat, newI = dpll(newS, newI)
    if sat == "Satisfacible":
        return sat, newI
    else:
        newS = eliminar_literal(S, lc)
        newI = extender_I(I, lc)
        return dpll(newS, newI)

---

In [159]:
from time import time
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

def obtiene_tiempos(fun, args, num_it=100):
    tiempos_fun = []
    for i in range(num_it):
        arranca = time()
        x = fun(*args)
        para = time()
        tiempos_fun.append(para - arranca)
    return tiempos_fun

def compara_entradas_funs(funs, nombres_funs, lista_args, N=100):
    entradas = []
    funcion = []
    tiempos = []
    lista_dfs = []
    for i, args in enumerate(lista_args):
        for j, fun in enumerate(funs):
            t = obtiene_tiempos(fun, [args], N)
            tiempos += t
            n = len(t)
            entradas += [i+1]*n
            funcion += [nombres_funs[j]]*n
        df = pd.DataFrame({'Long_entrada':entradas, 
                           'Funcion':funcion,
                           'Tiempo_prom':tiempos})
        lista_dfs.append(df)
    df = pd.concat(lista_dfs).reset_index()
    sns.lineplot(x='Long_entrada',y='Tiempo_prom',hue='Funcion',data=df)
    plt.show()

In [160]:
from random import uniform, choice

cantidad = 10
letras = [chr(i) for i in range(256, 256+cantidad)]
lista = [letras[0]]
formula = letras[0]
for p in letras[1:]:
            neg1 = '-' if uniform(0,1) > .5 else ''
            neg2 = '-' if uniform(0,1) > .5 else ''
            neg3 = '-' if uniform(0,1) > .5 else ''
            conectivo = choice(['Y','O','>'])
            if uniform(0,1) > .5:
                formula = neg1 + "(" + neg2 + formula + conectivo + neg3 + p + ")"
            else:
                formula = neg1 + "(" + neg2 + p + conectivo + neg3 + formula + ")"
            lista.append(formula)

In [165]:
nombres = ['dpll', 'walksat']
sat_dpll = lambda formula: dpll(tseitin(formula), {})
sat_walksat = lambda formula: walkSAT(tseitin(formula))
funs = [sat_dpll, sat_walksat]
compara_entradas_funs(funs, nombres, lista)

KeyboardInterrupt: 

Vemos que en todos los casos el DPLL es más rápido que el walkSAT.

---